<a href="https://colab.research.google.com/github/pduran5/ReportDocumentacio/blob/main/Report_Documentaci%C3%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report de Documentació del Drive del Departament d'Informàtica
* Variables globals a modificar:
  - **curs_actual** (p.ex. 2022-2023)
  - **trimestre** (1, 2, 3). Als trimestres 1 i 2 no es comprovaran projectes ni ASIX M10 que es fan al 3r trimestre
  - **start_date** (data d'inici) i **end_date** (data final) de la comprovació de modificació de fitxers. Ficar 1 dia més a la data final per a que comprovi el dia actual
  - **strict_naming** (a False no comprova les paraules exam i qual als fitxers lliurats)
  - **check_naming** (False no comprova que la nomenclatura estigui correcta)

La sortida és en Markdown. Convertir a .pdf amb la web https://md2pdf.netlify.app/







In [ ]:
import os, time
import unicodedata
from google.colab import drive
from datetime import datetime, timedelta

# Set timezone to Madrid
os.environ['TZ'] = 'Europe/Madrid'
time.tzset()

drive.mount('/content/gdrive', force_remount=True)

# VARIABLES GLOBALS
curs_actual = "2022-2023"
trimestre = 2 # Per al 1r i 2n trimestre no es comprovaran projectes ni ASIX M10
start_date = datetime(2022, 12, 14) # Última data fi: 31/03/2023
end_date = datetime(2023, 4, 1) # Ficar 1 dia més de l'actual per a que comprovi el dia d'avui
strict_naming = False
check_naming = True

curs_path = "/content/gdrive/MyDrive/Departament d\'Informàtica/Curs " + curs_actual

# FUNCIONS
def find_last_occurrence(s, c):
    last_pos = -1
    for i, x in enumerate(s):
        if x == c:
            last_pos = i
    return last_pos

def get_last_part(string, substring):
    substring_index = find_last_occurrence(string,substring)
    if substring_index != -1:
        return string[substring_index+len(substring):]
    else:
        return string

def find_and_return_suffix(s1, s2):
    if s2 in s1:
        suffix_start = s1.index(s2) + len(s2)
        return s1[suffix_start:]
    else:
        return None

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def detect_empty_modified_folders(root_dir, start_date, end_date, cadena):
    final_folders = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            modify_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            if start_date <= modify_time <= end_date:
                break
        else:
            if remove_accents(cadena) == remove_accents(get_last_part(dirpath, '/')):
              final_folders.append(dirpath)
    return final_folders

def modified_files_dates(root_dir, start_date, end_date):
    modified_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            modify_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            if start_date <= modify_time <= end_date:
                modified_files.append(file_path)
    return modified_files

def print_header():
    today = datetime.today()
    print('\n# REPORT DOCUMENTACIÓ DEPARTAMENT. DATA: ' + today.strftime('%d/%m/%Y') + ' ' + today.strftime('%H:%M'))

def check_modul(file):
    matchers = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm09', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9']
    for matcher in matchers:
        if matcher in file.lower():
            return True
    return False

def check_uf(file):
    matchers = ['uf1', 'uf2', 'uf3', 'uf4', 'uf5', 'uf6', 'uf01', 'uf02', 'uf03', 'uf04', 'uf05', 'uf06']
    for matcher in matchers:
        if matcher in file.lower():
            return True
    return False

def check_grup(file):
    matchers = ['smx1a', 'smx1b', 'smx1c', 'smx1d', 'smx2a', 'smx2b', 'smx2c', 'asix', 'dam', 'daw', 'damw', 'dawbi', 'asix', 'smx-1a', 'smx-1b', 'sxm-1c', 'sxm-1d', 'smx-2a', 'smx-2b', 'smx-2c', 'sxm1-a', 'smx1-b', 'sxm1-c', 'sxm1-d', 'smx2-a', 'smx2-b', 'smx2-c', 'grupa', 'grup1a', 'grupb', 'grup1b', 'grupc', 'grup1c', 'grupd', 'grup1d', 'grup2a', 'grup2b', 'grup2c', 'dax']
    for matcher in matchers:
        if matcher in file.lower():
            return True
    return False

def print_empty_folders(path, string):
    empty_folders = detect_empty_modified_folders(curs_path, start_date, end_date, string)
    for file in empty_folders:
        #if not(trimestre < 3 and ('M12 Sintesi' in remove_accents(file) or 'M12 Projecte' in file or 'M13 Projecte' in file or 'M14 Projecte' in file or 'M10 Administracio de SGBD' in remove_accents(file))):
            print('  - ❗ ' + find_and_return_suffix(file, path).replace('/', ' > '))

def check_examens():
    modified_files = modified_files_dates(curs_path + '/Exàmens i Qualificacions/', start_date, end_date)
    print('\n# EXÀMENS')

    # Exàmens no lliurat entre dates
    print('\n## 🔴 Exàmens no lliurats entre les dates {} i {}:'.format(start_date.date().strftime("%d/%m/%Y"), end_date.date().strftime("%d/%m/%Y")))
    print_empty_folders('i Qualificacions/', 'Exàmens')

    if check_naming == True:
      # Exàmens lliurats entre dates però amb nomenclatura incorrecta:
      print('\n## 🟡 Exàmens lliurats entre les dates {} i {} amb nomenclatura incorrecte:'.format(start_date.date().strftime("%d/%m/%Y"), end_date.date().strftime("%d/%m/%Y")))
    
      for file in modified_files:
          if '/Examens/' in remove_accents(file):
              arxiu = get_last_part(file, '/')

              if 'exam' in arxiu.lower() or strict_naming == False:
                  exam_found = True
              else:
                  exam_found = False

              modul_found = check_modul(arxiu)
              grup_found = check_grup(arxiu)
              uf_found = check_uf(arxiu)
            
              if exam_found==False or modul_found==False or grup_found==False or uf_found==False:
                  output = '  - ⚠️ ' + find_and_return_suffix(file, 'i Qualificacions/') + ' ➡️ **Falta: '
                  pendent = []

                  if exam_found == False:
                    pendent.append('exam')
                  if modul_found == False:
                      pendent.append('mòdul')
                  if grup_found == False:
                      pendent.append('grup')
                  if uf_found == False:
                      pendent.append('UF')

                  output = output.replace('/', ' > ')
                  print(output + ", ".join(pendent) + '**')

def check_qualificacions():
    modified_files = modified_files_dates(curs_path + '/Exàmens i Qualificacions/', start_date, end_date)
    print('\n# QUALIFICACIONS')

    # Qualificacions no lliurades entre dates
    print('\n## 🔴 Qualificacions no lliurades entre les dates {} i {}:'.format(start_date.date().strftime("%d/%m/%Y"), end_date.date().strftime("%d/%m/%Y")))
    print_empty_folders('i Qualificacions/', 'Qualificacions')

    if check_naming == True:
      # Qualificacions lliurades entre dates però amb nomenclatura incorrecta:
      print('\n## 🟡 Qualificacions lliurades entre les dates {} i {} amb nomenclatura incorrecte:'.format(start_date.date().strftime("%d/%m/%Y"), end_date.date().strftime("%d/%m/%Y")))
    
      for file in modified_files:
          if '/Qualificacions/' in remove_accents(file):
              arxiu = get_last_part(file, '/')

              if 'qual' in arxiu.lower() or strict_naming == False:
                  qual_found = True
              else:
                  qual_found = False

              modul_found = check_modul(arxiu)
              grup_found = check_grup(arxiu)
            
              if qual_found==False or modul_found==False or grup_found==False:
                  output = '  - ⚠️ ' + find_and_return_suffix(file, 'i Qualificacions/') + ' ➡️ **Falta: '
                  pendent = []

                  if qual_found == False:
                      pendent.append('qual')
                  if modul_found == False:
                      pendent.append('mòdul')
                  if grup_found == False:
                      pendent.append('grup')

                  output = output.replace('/', ' > ')
                  print(output + ", ".join(pendent) + '**')

def check_array(arr, char):
    result = []
    for item in arr:
        if item != char:
            result.append(item)
    return ", ".join(result).upper() if result else None

def check_programacions():
    print('\n# PROGRAMACIONS')

    # Arrays per cicle per fer el check del lliurament, s'actualizarà amb x al mòdul de la programació que es trobi lliurada
    smx = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm12', 'm13']
    asix = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm09', 'm10', 'm11', 'm14', 'm15']
    dam = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm09', 'm10', 'm13', 'm14']
    daw = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm09', 'm12', 'm13']
    dawbio = ['m01', 'm02', 'm03', 'm04', 'm05', 'm06', 'm07', 'm08', 'm09', 'm12', 'm13', 'm14']
    
    programacions_path = curs_path + '/Programacions'
    for dirpath, dirnames, filenames in os.walk(programacions_path):
        for file in filenames:
            if 'Seguiments de la Program' not in file:
                arxiu = get_last_part(file, '/')

                if 'smx' in arxiu.lower():
                    for i, item in enumerate(smx):
                        if item in arxiu.lower() and 'prog' in arxiu.lower():
                            smx[i] = 'x'

                if 'asix' in arxiu.lower():
                    for i, item in enumerate(asix):
                        if item in arxiu.lower() and 'prog' in arxiu.lower():
                            asix[i] = 'x'

                if 'dam' in arxiu.lower():
                    for i, item in enumerate(dam):
                        if item in arxiu.lower() and 'prog' in arxiu.lower():
                            dam[i] = 'x'

                if 'daw' in arxiu.lower():
                    for i, item in enumerate(daw):
                        if item in arxiu.lower() and 'prog' in arxiu.lower():
                            daw[i] = 'x'

                if 'dawbio' in arxiu.lower():
                    for i, item in enumerate(dawbio):
                        if item in arxiu.lower() and 'prog' in arxiu.lower():
                            dawbio[i] = 'x'
    
    if check_array(smx, 'x') == None:
        print('  - 🟢 SMX: Totes les programacions lliurades')
    else:
        print('  - 🔴 SMX: Programacions pendents de lliurar: ' + check_array(smx, 'x'))

    if check_array(asix, 'x') == None:
        print('  - 🟢 ASIX: Totes les programacions lliurades')
    else:
        print('  - 🔴 ASIX: Programacions pendents de lliurar: ' + check_array(asix, 'x'))

    if check_array(dam, 'x') == None:
        print('  - 🟢 DAM: Totes les programacions lliurades')
    else:
        print('  - 🔴 DAM: Programacions pendents de lliurar: ' + check_array(dam, 'x'))
    
    if check_array(daw, 'x') == None:
        print('  - 🟢 DAW: Totes les programacions lliurades')
    else:
        print('  - 🔴 DAW: Programacions pendents de lliurar: ' + check_array(daw, 'x'))
    
    if check_array(dawbio, 'x') == None:
        print('  - 🟢 DAWBIO: Totes les programacions lliurades')
    else:
        print('  - 🔴 DAWBIO: Programacions pendents de lliurar: ' + check_array(dawbio, 'x'))

def check_harmonitzacions():
    modified_files = modified_files_dates(curs_path + '/Harmonitzacions/', start_date, end_date)
    all_files = modified_files_dates(curs_path + '/Harmonitzacions/', datetime(2000, 1, 1), end_date)
    
    for i, item in enumerate(all_files):
      for modified in modified_files:
        if modified in item:
          all_files[i] = 'x'

    print('\n# HARMONITZACIONS')

    # Harmonitzacions no actualitzades entre dates
    print('\n## 🔴 Harmonitzacions no actualitzades entre les dates {} i {}:'.format(start_date.date().strftime("%d/%m/%Y"), end_date.date().strftime("%d/%m/%Y")))
    for file in all_files:
      if file != 'x':
        print('  - ❗ ' + find_and_return_suffix(file, 'Harmonitzacions/'))

# INICI PROGRAMA
print_header()

check_programacions()
check_examens()
check_qualificacions()
check_harmonitzacions()

Mounted at /content/gdrive

# REPORT DOCUMENTACIÓ DEPARTAMENT. DATA: 31/03/2023 12:53

# PROGRAMACIONS
  - 🟢 SMX: Totes les programacions lliurades
  - 🟢 ASIX: Totes les programacions lliurades
  - 🟢 DAM: Totes les programacions lliurades
  - 🟢 DAW: Totes les programacions lliurades
  - 🟢 DAWBIO: Totes les programacions lliurades

# EXÀMENS

## 🔴 Exàmens no lliurats entre les dates 14/12/2022 i 01/04/2023:
  - ❗ CFGM SMX > M02 Sistemes operatius monolloc > SMX1B > Exàmens
  - ❗ CFGM SMX > M03 Aplicacions ofimàtiques > SMX1A > Exàmens
  - ❗ CFGS DAWBIO > M09 Disseny d'interfícies web > Exàmens
  - ❗ CFGS ASIX > M10 Administració de SGBD > Exàmens
  - ❗ CFGS DAW > M01 Sistemes informàtics > Exàmens

## 🟡 Exàmens lliurats entre les dates 14/12/2022 i 01/04/2023 amb nomenclatura incorrecte:
  - ⚠️ CFGM SMX > M02 Sistemes operatius monolloc > SMX1C > Exàmens > exam_smx_m02_uf1_examen2_teo_prac_2022 (1).odt ➡️ **Falta: grup**
  - ⚠️ CFGM SMX > M02 Sistemes operatius monolloc > SMX1